In [1]:
import pandas as pd

In [3]:
ids = pd.read_csv('data/data for graph analysis/graph skid list.csv')

In [12]:
ids = ids.rename(columns={'n.SK_ID_CURR':'SK_ID_CURR'})
ids.head(1)

,SK_ID_CURR
0,101156


In [17]:
train_data = pd.read_csv('result_data/train_app_prev_b_pos_merged.csv')

In [20]:
merged=pd.merge(ids,train_data,on='SK_ID_CURR')

In [24]:
merged = merged.sort_values(['SK_ID_CURR'],axis=0)

In [26]:
merged.head(1)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,client_cash_SK_DPD_DEF_min_count,client_cash_SK_DPD_DEF_min_mean,client_cash_SK_DPD_DEF_min_max,client_cash_SK_DPD_DEF_min_min,client_cash_SK_DPD_DEF_min_sum,client_cash_SK_DPD_DEF_sum_count,client_cash_SK_DPD_DEF_sum_mean,client_cash_SK_DPD_DEF_sum_max,client_cash_SK_DPD_DEF_sum_min,client_cash_SK_DPD_DEF_sum_sum
999,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0


In [28]:
merged.to_csv('data/data for graph analysis/1000_train_graph.csv',index=False)

In [29]:
# prediction

In [2]:
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [3]:
params = {}
params['learning_rate'] = 0.09043007325486686
#params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 45
params['min_data'] = 50
params['max_depth'] = 10

In [41]:
X = merged
Y  = X['TARGET']
X  = X.drop(columns=['TARGET','SK_ID_CURR'],axis=1)

In [42]:
X.head(1)

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,client_cash_SK_DPD_DEF_min_count,client_cash_SK_DPD_DEF_min_mean,client_cash_SK_DPD_DEF_min_max,client_cash_SK_DPD_DEF_min_min,client_cash_SK_DPD_DEF_min_sum,client_cash_SK_DPD_DEF_sum_count,client_cash_SK_DPD_DEF_sum_mean,client_cash_SK_DPD_DEF_sum_max,client_cash_SK_DPD_DEF_sum_min,client_cash_SK_DPD_DEF_sum_sum
999,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,9461,...,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [44]:
d_train = lgb.Dataset(X_train, label=y_train)

In [45]:
clf = lgb.train(params, d_train, 200)

In [46]:
y_pred=clf.predict(X_test)

In [47]:
type(y_test.values)
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [48]:
auc(fpr, tpr)

0.773936170212766

In [ ]:
#0.773936170212766

In [4]:
# merging both:
graph = pd.read_csv('data/data for graph analysis/from graph 895.csv')

In [5]:
data = pd.read_csv('data/data for graph analysis/1000_train_graph.csv')

In [6]:
final = pd.merge(graph,data,on='SK_ID_CURR')

In [7]:
final.head()

,SK_ID_CURR,TotalInstallments,DaysInstallMinusEntry,AmtPaymentByInstallment,TotalBureau,TotalActiveBureau,ActiveBureauByTotalBureauRatio,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,...,client_cash_SK_DPD_DEF_min_count,client_cash_SK_DPD_DEF_min_mean,client_cash_SK_DPD_DEF_min_max,client_cash_SK_DPD_DEF_min_min,client_cash_SK_DPD_DEF_min_sum,client_cash_SK_DPD_DEF_sum_count,client_cash_SK_DPD_DEF_sum_mean,client_cash_SK_DPD_DEF_sum_max,client_cash_SK_DPD_DEF_sum_min,client_cash_SK_DPD_DEF_sum_sum
0,100191,1,84,1,0,0,NaN,0,0,1,...,34.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0
1,100191,1,21,1,0,0,NaN,0,0,1,...,34.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0
2,100452,1,0,1,0,0,NaN,0,0,1,...,61.0,0.0,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0
3,100452,2,9,1,0,0,NaN,0,0,1,...,61.0,0.0,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0
4,100452,1,-3,1,0,0,NaN,0,0,1,...,61.0,0.0,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0


In [8]:
X = final
Y  = X['TARGET']
X  = X.drop(columns=['TARGET','SK_ID_CURR'],axis=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
d_train = lgb.Dataset(X_train, label=y_train)

In [15]:
clf2 = lgb.train(params, d_train, 200)

In [16]:
y_pred=clf2.predict(X_test)

In [17]:
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [18]:
auc(fpr, tpr)

0.9351297405189621